In [2]:
# evaluate multinomial logistic regression model
from numpy import mean
from numpy import std
import os
import pandas as pd
import numpy as np 
import math
import json
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings("ignore")

In [15]:
dataset_name = 'nlsw'
i=3
pd.read_csv(dataset_name+'/'+dataset_name+'_stage_'+str(i+1)+'.csv', index_col='person_id').drop('Unnamed: 0', axis=1).dropna()

,single_couple,couple_single,single_single,childbearing_no_yes,childbearing_yes_no,has_migration,education_level,first_job_income_quantile,num_of_event,event_weighted_score,event_entropy_score,prev_stage,gender,birth_year,race,dv_stage
person_id,,,,,,,,,,,,,,,,
1,1,1,1,0,0,0,high-school,3,5,3.4,1.609438,s_cluster_6,female,1951.0,black,s_cluster_6
2,1,0,0,0,0,0,high-school,1,3,2.1,1.098612,s_cluster_2,female,1951.0,black,s_cluster_2
3,0,0,0,0,0,0,high-school,4,2,1.3,0.693147,s_cluster_5,female,1945.0,black,s_cluster_2
5,1,0,0,1,0,0,high-school,5,4,2.6,1.386294,s_cluster_5,female,1945.0,white,s_cluster_5
6,1,1,0,1,0,0,high-school,4,6,3.9,1.560710,s_cluster_2,female,1946.0,white,s_cluster_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5143,1,0,0,1,0,0,high-school,1,4,2.6,1.386294,s_cluster_1,female,1950.0,black,s_cluster_1
5146,1,0,0,1,1,0,high-school,2,5,2.7,1.609438,s_cluster_1,female,1951.0,white,s_cluster_1
5152,1,0,0,1,0,0,high-school,1,4,2.6,1.386294,s_cluster_5,female,1952.0,white,s_cluster_5


In [3]:
os.chdir(r"D:\A李杜若\留学\1-申请阶段\科研\可视化_王懿芳学姐\CFPS\NLS\3-regression-model")

In [1]:
def reg_backend(dataset_name, group_name, model_param, ref_groups, event_weight): #
# 有用的参数
    num_var_list = ['num_of_event', 'event_weighted_score','event_entropy_score','birth_year']
    num_change_var_list = ['num_of_event', 'event_weighted_score','event_entropy_score']
    num_control_var_list = ['birth_year']
    if group_name == "education":
        cat_var_list = ['first_job_income_quantile','prev_stage', 'gender', 'race']
        cat_change_var_list = ['first_job_income_quantile','prev_stage']
    else:
        cat_var_list = ['education_level', 'first_job_income_quantile','prev_stage', 'gender', 'race']
        cat_change_var_list = ['education_level', 'first_job_income_quantile','prev_stage']
    cat_control_var_list = [ 'gender', 'race']
    if dataset_name == 'nlsy':
        bool_var_list = ['single_couple', 'couple_single', 'single_single', 'childbearing_no_yes', 'childbearing_yes_no', 'has_migration','noncentral_central', 'central_noncentral',] # nlsy migration
    else:
        bool_var_list = ['single_couple', 'couple_single', 'single_single', 'childbearing_no_yes', 'childbearing_yes_no', 'has_migration',]
    dependent_var = ['dv_stage']
    control_var_list = ['gender','race','birth_year']
    control_dict = {    
        'gender': ['male','female'],
        'race': ['white','black','other']}


    dataset_info = {
    'stage_n':{
        'nlsw':7,
        'nlsm':3,
        'nlsy':8,
        'psid':10,
    },
    'cluster_n':{
        'nlsw':9,
        'nlsm':5,
        'nlsy':9,
        'psid':10,   
    },
}
    cats_dict = {
    'nlsw': {
    'education_level': ['high-school','college'], # nlsw,nlsm 1,2; nlsy, nlsw 0,1,2
    'first_job_income_quantile': [i+1 for i in range(5)],
    'prev_stage': [f's_cluster_{i}' for i in range(dataset_info['cluster_n']['nlsw'])],
    },
    'nlsm': {
    'education_level': ['high-school','college'], # nlsw,nlsm 1,2; nlsy, nlsw 0,1,2
    'first_job_income_quantile': [i+1 for i in range(5)],
    'prev_stage': [f's_cluster_{i}' for i in range(dataset_info['cluster_n']['nlsm'])],
    },
    'nlsy': {
    'education_level': ['high-school-noncompletion', 'high-school', 'college'], # nlsw,nlsm 1,2; nlsy, nlsw 0,1,2
    'first_job_income_quantile': [i+1 for i in range(5)],
    'prev_stage': [f's_cluster_{i}' for i in range(dataset_info['cluster_n']['nlsy'])],
    },
    'psid': {
    'education_level': ['high-school-noncompletion', 'high-school', 'college'], # nlsw,nlsm 1,2; nlsy, psid 0,1,2
    'first_job_income_quantile': [i+1 for i in range(5)],
    'prev_stage': [f's_cluster_{i}' for i in range(dataset_info['cluster_n']['psid'])],
    },
}
    # 设置储存数据的dicts
    ## 总dict
    reg_dict = {
        "lrt":{},
        "coef_p":{},
    }
    
    drop_var_list = bool_var_list+cat_change_var_list+num_change_var_list
    ### lrt dict
    LR_dict = dict(zip(['full_model']+drop_var_list, [{'loglikelihood':dict(zip(['stage'+str(i+1) for i in range(1,dataset_info['stage_n'][dataset_name])], [0 for i in range(1,dataset_info['stage_n'][dataset_name])])),'degree_of_freedom':0}]+[{'LR': dict(zip(['stage'+str(i+1) for i in range(1,dataset_info['stage_n'][dataset_name])], [0 for i in range(1,dataset_info['stage_n'][dataset_name])])), 'degree_of_freedom': 0} for i in range(len(drop_var_list))]))
    
    ### coef dict
    ref_dv = ref_groups['dv_stage']
    dvs = ['dv_'+dv for dv in [cl for cl in cats_dict[dataset_name]["prev_stage"] if cl!=ref_dv]]
    stages = [[] for t in range(dataset_info['cluster_n'][dataset_name]-1)]
    coef_p_dict = dict(zip(dvs,stages))
    
    # 循环各个stage的数据集，进行模型拟合
    for i in range(1,dataset_info['stage_n'][dataset_name]):
        # 读取数据
        df0 = pd.read_csv(dataset_name+'/'+dataset_name+'_stage_'+str(i+1)+'.csv', index_col='person_id').drop('Unnamed: 0', axis=1).dropna()
        input_X = df0.iloc[:,:-1]
        input_y = df0['dv_stage']
        

        # 设定iv的ref   
        if group_name == "education":
            choice_order_list = ['first_job_income_quantile','prev_stage']
        else:
            choice_order_list = ['education_level', 'first_job_income_quantile','prev_stage']
        cat_dict = cats_dict[dataset_name]
        for var in choice_order_list:
            choice = ref_groups[var]
            l0 = cat_dict[var]
            l0.remove(choice)
            cat_dict[var] = [choice]+l0
            input_X[var]=pd.Categorical(input_X[var], categories=cat_dict[var])
        # 设定dv的ref
        ref_dv = ref_groups['dv_stage']
        input_y = df0["dv_stage"].replace({ref_dv: 'a_'+ref_dv})
        dv_rest_list = [f's_cluster_{i}' for i in range(dataset_info['cluster_n'][dataset_name])]
        dv_rest_list.remove(ref_dv) 
        
        # 训练模型
        cat_dummies = pd.get_dummies(input_X[cat_var_list], drop_first=True)
        train_X = pd.concat([input_X[num_var_list+bool_var_list], cat_dummies], axis=1)
        sc_X = StandardScaler()
        train_X_sd = pd.DataFrame(sc_X.fit_transform(train_X), index=train_X.index, columns=train_X.columns)
        train_y = input_y
        mlogit_mod = sm.MNLogit(train_y, train_X_sd)
        mlogit_res = mlogit_mod.fit(method='bfgs')
        parameters, std_error, loglikelihood  = mlogit_res.params, mlogit_res.bse, mlogit_res.llf
        p_values, t_values = mlogit_res.pvalues, mlogit_res.tvalues
        
        # 输出到指定格式
        ## lrt
        full_llf  = mlogit_res.llf
        full_df = mlogit_res.df_model
        LR_dict['full_model']['degree_of_freedom']=full_df
        LR_dict['full_model']['loglikelihood']['stage'+str(i+1)] = full_llf    
        

        
        for var in drop_var_list:
            cat_var_list1=[k for k in cat_var_list if k!=var]
            num_var_list1=[k for k in num_var_list if k!=var]
            bool_var_list1=[k for k in bool_var_list if k!=var]

            cat_dummies1 = pd.get_dummies(input_X[cat_var_list1], drop_first=True)
            train_X_drop = pd.concat([input_X[num_var_list1+bool_var_list1], cat_dummies1], axis=1)
            sc_X = StandardScaler()
            train_X_drop_sd = pd.DataFrame(sc_X.fit_transform(train_X_drop), index=train_X_drop.index, columns=train_X_drop.columns)

            train_y = input_y
            mlogit_mod = sm.MNLogit(train_y, train_X_drop_sd)
            mlogit_res = mlogit_mod.fit(method='bfgs')
            var_llf  = mlogit_res.llf
            var_df = mlogit_res.df_model
            
            LR = 2*(full_llf-var_llf)
            df = full_df-var_df

            LR_dict[var]['degree_of_freedom'] = df
            LR_dict[var]['LR']['stage'+str(i+1)] = LR
        
        
        ## coef_p
        for j in range(parameters.shape[1]):
            dict_s = {"stage_name":'', "data":{}}
            dict_s["stage_name"]="stage"+str(i+1)
            # bool & num
            l0=[p_values[j], parameters[j], parameters[j].apply(lambda x: math.exp(x))]
            df = pd.concat(l0, axis=1)
            df.columns = ['p-value', 'coef', 'e_coef'] 
            dict_s["data"] = df.loc[num_var_list+bool_var_list].to_dict('index')
            # categories
            for idx, ch_cat in enumerate(choice_order_list):
                names = [k for k in list(df.index) if k.startswith(ch_cat)]
                df1 = df.loc[names]
                df1.index = [k.replace(ch_cat, str(ref_groups[ch_cat])) for k in names]
                dict_s["data"][ch_cat] = df1.to_dict('index')
            # append list
            dv_j = 'dv_'+dv_rest_list[j]
            coef_p_dict[dv_j].append(dict_s)
            
    reg_dict["coef_p"] = coef_p_dict
    reg_dict["lrt"] = LR_dict 
    
    return reg_dict

In [ ]:
reg_dict = reg_backend(dataset_name='nlsw', group_name="female", model_param=0,  event_weight=0,ref_groups={
        "education_level": "high-school", 
        "first_job_income_quantile": 1,
        "prev_stage": "s_cluster_1", 
        "dv_stage": "s_cluster_1",
      })

         Current function value: 1.823021
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.824475
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.827205
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.823725
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.824039
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.827892
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.824142
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.856966
         Iterations:

In [ ]:
reg_dict

{'lrt': {'full_model': {'loglikelihood': {'stage2': -3206.6944691437657,
    'stage3': -3125.5622161395636,
    'stage4': -2848.6117360031963,
    'stage5': -2731.089223693581,
    'stage6': -2405.1245526148414,
    'stage7': -2171.3088182331608},
   'degree_of_freedom': 192.0},
  'single_couple': {'LR': {'stage2': 5.113389367210402,
    'stage3': 1.744156906211174,
    'stage4': 8.190710688868421,
    'stage5': 2.7639354593429744,
    'stage6': 9.127933046627732,
    'stage7': 4.176241550998384},
   'degree_of_freedom': 8.0},
  'couple_single': {'LR': {'stage2': 14.716678226172007,
    'stage3': 4.706360750385102,
    'stage4': 9.48810207879069,
    'stage5': 0.9538733696508643,
    'stage6': 2.203822633568052,
    'stage7': 7.5893336177732635},
   'degree_of_freedom': 8.0},
  'single_single': {'LR': {'stage2': 2.477333448077843,
    'stage3': 5.085951545822354,
    'stage4': 4.548535185829678,
    'stage5': 1.753905558394763,
    'stage6': 9.738360942738836,
    'stage7': 5.922897787

### 毕设版本

In [4]:
## 为了毕设：仅输出模型，不整理成json
def reg_backend(dataset_name, group_name, model_param, ref_groups, event_weight): #
# 有用的参数
    num_var_list = ['num_of_event', 'event_weighted_score','event_entropy_score','birth_year']
    num_change_var_list = ['num_of_event', 'event_weighted_score','event_entropy_score']
    num_control_var_list = ['birth_year']
    if group_name == "education":
        cat_var_list = ['first_job_income_quantile','prev_stage', 'gender', 'race']
        cat_change_var_list = ['first_job_income_quantile','prev_stage']
    else:
        cat_var_list = ['education_level', 'first_job_income_quantile','prev_stage', 'gender', 'race']
        cat_change_var_list = ['education_level', 'first_job_income_quantile','prev_stage']
    cat_control_var_list = [ 'gender', 'race']
    if dataset_name == 'nlsy':
        bool_var_list = ['single_couple', 'couple_single', 'single_single', 'childbearing_no_yes', 'childbearing_yes_no', 'has_migration','noncentral_central', 'central_noncentral',] # nlsy migration
    else:
        bool_var_list = ['single_couple', 'couple_single', 'single_single', 'childbearing_no_yes', 'childbearing_yes_no', 'has_migration',]
    dependent_var = ['dv_stage']
    control_var_list = ['gender','race','birth_year']
    control_dict = {    
        'gender': ['male','female'],
        'race': ['white','black','other']}


    dataset_info = {
    'stage_n':{
        'nlsw':7,
        'nlsm':3,
        'nlsy':8,
        'psid':10,
    },
    'cluster_n':{
        'nlsw':9,
        'nlsm':5,
        'nlsy':9,
        'psid':10,   
    },
}
    cats_dict = {
    'nlsw': {
    'education_level': ['high-school','college'], # nlsw,nlsm 1,2; nlsy, nlsw 0,1,2
    'first_job_income_quantile': [i+1 for i in range(5)],
    'prev_stage': [f's_cluster_{i}' for i in range(dataset_info['cluster_n']['nlsw'])],
    },
    'nlsm': {
    'education_level': ['high-school','college'], # nlsw,nlsm 1,2; nlsy, nlsw 0,1,2
    'first_job_income_quantile': [i+1 for i in range(5)],
    'prev_stage': [f's_cluster_{i}' for i in range(dataset_info['cluster_n']['nlsm'])],
    },
    'nlsy': {
    'education_level': ['high-school-noncompletion', 'high-school', 'college'], # nlsw,nlsm 1,2; nlsy, nlsw 0,1,2
    'first_job_income_quantile': [i+1 for i in range(5)],
    'prev_stage': [f's_cluster_{i}' for i in range(dataset_info['cluster_n']['nlsy'])],
    },
    'psid': {
    'education_level': ['high-school-noncompletion', 'high-school', 'college'], # nlsw,nlsm 1,2; nlsy, psid 0,1,2
    'first_job_income_quantile': [i+1 for i in range(5)],
    'prev_stage': [f's_cluster_{i}' for i in range(dataset_info['cluster_n']['psid'])],
    },
}
    # 设置储存数据的dicts
    ## 总dict
    reg_dict = {
        "lrt":{},
        "coef_p":{},
    }
    
    drop_var_list = bool_var_list+cat_change_var_list+num_change_var_list
    ### lrt dict
    LR_dict = dict(zip(['full_model']+drop_var_list, [{'loglikelihood':dict(zip(['stage'+str(i+1) for i in range(1,dataset_info['stage_n'][dataset_name])], [0 for i in range(1,dataset_info['stage_n'][dataset_name])])),'degree_of_freedom':0}]+[{'LR': dict(zip(['stage'+str(i+1) for i in range(1,dataset_info['stage_n'][dataset_name])], [0 for i in range(1,dataset_info['stage_n'][dataset_name])])), 'degree_of_freedom': 0} for i in range(len(drop_var_list))]))
    
    ### coef dict
    ref_dv = ref_groups['dv_stage']
    dvs = ['dv_'+dv for dv in [cl for cl in cats_dict[dataset_name]["prev_stage"] if cl!=ref_dv]]
    stages = [[] for t in range(dataset_info['cluster_n'][dataset_name]-1)]
    coef_p_dict = dict(zip(dvs,stages))
    
    # 循环各个stage的数据集，进行模型拟合
    for i in range(1,dataset_info['stage_n'][dataset_name]):
        # 读取数据
        df0 = pd.read_csv(dataset_name+'/'+dataset_name+'_stage_'+str(i+1)+'.csv', index_col='person_id').drop('Unnamed: 0', axis=1).dropna()
        input_X = df0.iloc[:,:-1]
        input_y = df0['dv_stage']
        

        # 设定iv的ref   
        if group_name == "education":
            choice_order_list = ['first_job_income_quantile','prev_stage']
        else:
            choice_order_list = ['education_level', 'first_job_income_quantile','prev_stage']
        cat_dict = cats_dict[dataset_name]
        for var in choice_order_list:
            choice = ref_groups[var]
            l0 = cat_dict[var]
            l0.remove(choice)
            cat_dict[var] = [choice]+l0
            input_X[var]=pd.Categorical(input_X[var], categories=cat_dict[var])
        # 设定dv的ref
        ref_dv = ref_groups['dv_stage']
        input_y = df0["dv_stage"].replace({ref_dv: 'a_'+ref_dv})
        dv_rest_list = [f's_cluster_{i}' for i in range(dataset_info['cluster_n'][dataset_name])]
        dv_rest_list.remove(ref_dv) 
        
        # 训练模型
        cat_dummies = pd.get_dummies(input_X[cat_var_list], drop_first=True)
        train_X = pd.concat([input_X[num_var_list+bool_var_list], cat_dummies], axis=1)
        sc_X = StandardScaler()
        train_X_sd = pd.DataFrame(sc_X.fit_transform(train_X), index=train_X.index, columns=train_X.columns)
        train_y = input_y
        mlogit_mod = sm.MNLogit(train_y, train_X_sd)
        mlogit_res = mlogit_mod.fit(method='bfgs')
        parameters, std_error, loglikelihood  = mlogit_res.params, mlogit_res.bse, mlogit_res.llf
        p_values, t_values = mlogit_res.pvalues, mlogit_res.tvalues
        
        output = mlogit_res
    #     # 输出到指定格式
    #     ## lrt
    #     full_llf  = mlogit_res.llf
    #     full_df = mlogit_res.df_model
    #     LR_dict['full_model']['degree_of_freedom']=full_df
    #     LR_dict['full_model']['loglikelihood']['stage'+str(i+1)] = full_llf    
        

        
    #     for var in drop_var_list:
    #         cat_var_list1=[k for k in cat_var_list if k!=var]
    #         num_var_list1=[k for k in num_var_list if k!=var]
    #         bool_var_list1=[k for k in bool_var_list if k!=var]

    #         cat_dummies1 = pd.get_dummies(input_X[cat_var_list1], drop_first=True)
    #         train_X_drop = pd.concat([input_X[num_var_list1+bool_var_list1], cat_dummies1], axis=1)
    #         sc_X = StandardScaler()
    #         train_X_drop_sd = pd.DataFrame(sc_X.fit_transform(train_X_drop), index=train_X_drop.index, columns=train_X_drop.columns)

    #         train_y = input_y
    #         mlogit_mod = sm.MNLogit(train_y, train_X_drop_sd)
    #         mlogit_res = mlogit_mod.fit(method='bfgs')
    #         var_llf  = mlogit_res.llf
    #         var_df = mlogit_res.df_model
            
    #         LR = 2*(full_llf-var_llf)
    #         df = full_df-var_df

    #         LR_dict[var]['degree_of_freedom'] = df
    #         LR_dict[var]['LR']['stage'+str(i+1)] = LR
        
        
    #     ## coef_p
    #     for j in range(parameters.shape[1]):
    #         dict_s = {"stage_name":'', "data":{}}
    #         dict_s["stage_name"]="stage"+str(i+1)
    #         # bool & num
    #         l0=[p_values[j], parameters[j], parameters[j].apply(lambda x: math.exp(x))]
    #         df = pd.concat(l0, axis=1)
    #         df.columns = ['p-value', 'coef', 'e_coef'] 
    #         dict_s["data"] = df.loc[num_var_list+bool_var_list].to_dict('index')
    #         # categories
    #         for idx, ch_cat in enumerate(choice_order_list):
    #             names = [k for k in list(df.index) if k.startswith(ch_cat)]
    #             df1 = df.loc[names]
    #             df1.index = [k.replace(ch_cat, str(ref_groups[ch_cat])) for k in names]
    #             dict_s["data"][ch_cat] = df1.to_dict('index')
    #         # append list
    #         dv_j = 'dv_'+dv_rest_list[j]
    #         coef_p_dict[dv_j].append(dict_s)
            
    # reg_dict["coef_p"] = coef_p_dict
    # reg_dict["lrt"] = LR_dict 
    
    return output

In [7]:
output = reg_backend(dataset_name='nlsy', group_name="female", model_param=0,  event_weight=0,ref_groups={
        "education_level": "college", 
        "first_job_income_quantile": 3,
        "prev_stage": "s_cluster_0", 
        "dv_stage": "s_cluster_0",
      })

         Current function value: 1.753040
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.551524
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.430938
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.285064
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.268038
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.214774
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
         Current function value: 1.153685
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36


In [6]:
pd.DataFrame(output.summary().tables[1]).to_csv('thesis_nlsy.csv')